In [ ]:
from datacube import Datacube
from datacube.virtual.impl import VirtualDatasetBox
from datacube.virtual import construct
from datacube.utils.geometry import CRS, Geometry
from datacube.utils.geometry.gbox import GeoboxTiles
import boto3
import os
import fiona
import yaml
import numpy as np
import pandas as pd
from datetime import datetime
from shapely import ops, geometry
import pyproj
from multiprocessing import Pool, Process

In [ ]:
from datacube_wps.processes.witprocess import *
from dea_tools.dask import create_dask_gateway_cluster
#from datacube.utils.aws import configure_s3_access

In [ ]:
import awswrangler as wr

In [ ]:
from dask_gateway import Gateway
gateway = Gateway()
cluster_names = gateway.list_clusters()
print("cluster still runing", cluster_names)
for n in cluster_names:
    cluster = gateway.connect(n.name)
    cluster.shutdown()

In [ ]:
options = gateway.cluster_options()

options['profile'] = 'r5_L'
options['jupyterhub_user'] = os.getenv('JUPYTERHUB_USER')

cluster = gateway.new_cluster(options)

cluster.adapt(minimum=1, maximum=10)

cluster

In [ ]:
client = cluster.get_client()

In [ ]:
client

In [ ]:
client.close()

In [ ]:
session = boto3.Session(profile_name='dev')
landsat_shapefile = '../auxfiles/landsat_au/landsat_au.shp'

In [ ]:
def shape_list(shapefile):
    with fiona.open(shapefile) as allshapes:
        for shape in allshapes:
            yield(shape)

In [ ]:
def intersect_landsat_pathrow(shapefile_name, landsat_shapefile):
    output_list = []
    project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3577", always_xy=True).transform
    for input_shape in shape_list(shapefile_name):
        non_contain = True
        input_poly = geometry.shape(input_shape['geometry'])
        for l_shape in shape_list(landsat_shapefile):
            overlap = ops.transform(project,  geometry.shape(l_shape['geometry'])).intersection(input_poly)
            if overlap.area > 0:
                if overlap.area / input_poly.area > 0.9:
                    non_contain = False
                    output_list += [(input_shape, 1)]
        if non_contain:
            output_list += [(input_shape, 15)]
    return output_list

In [ ]:
shapefile_name = './shapefiles/Lock_4_Floodplain_25062021.shp'
wit_path = 's3://dea-wit-dev/c3-samples-3577/Lock_4_Floodplain_25062021/'
# shapefile_name = 'qld_shapefile/Queensland_dominant_wetland_areas_22042020.shp'

In [ ]:
shape_set = intersect_landsat_pathrow(shapefile_name, landsat_shapefile)

In [ ]:
from wit_compute import compute_wit
import time

In [ ]:
def first_process():
    print("do nothing")
    return

In [ ]:
# run this if parallel
if __name__ == '__main__':
    p_list = []
    p = Process(target=first_process)
    p.start()
    p_list += [p]
    for i in range(len(shape_set)):
        p = Process(target=compute_wit, args=(shape_set[i], ('1987-01-01', '2022-01-01'), wit_path))
        p.start()
        p_list += [p]
    
    print(p_list)
    for p in p_list:
        print(p)
        p.join()

In [ ]:
# run this for a single polygon
compute_wit(shape_set[5], ('1987-01-01', '2022-01-01'), wit_path)